In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# tf.keras.backend.clear_session()

In [1]:
from classification_models.tfkeras import Classifiers
import tensorflow as tf
import tensorflow_datasets as tfds
keras = tf.keras

In [2]:
print(tfds.list_builders())

['bair_robot_pushing_small', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cifar10', 'cifar100', 'coco2014', 'diabetic_retinopathy_detection', 'dummy_dataset_shared_generator', 'dummy_mnist', 'fashion_mnist', 'image_label_folder', 'imagenet2012', 'imdb_reviews', 'lm1b', 'lsun', 'mnist', 'moving_mnist', 'nsynth', 'omniglot', 'open_images_v4', 'quickdraw_bitmap', 'squad', 'starcraft_video', 'svhn_cropped', 'tf_flowers', 'wmt_translate_ende', 'wmt_translate_enfr']


In [30]:
train_dataset, dataset_info = tfds.load(name="fashion_mnist", split=tfds.Split.TRAIN, as_supervised=True, with_info=True)
test_dataset = tfds.load(name="fashion_mnist", split=tfds.Split.TEST, as_supervised=True)

In [31]:
train_dataset = train_dataset.batch(64) 
test_dataset = test_dataset.batch(64)
train_dataset

<DatasetV1Adapter shapes: ((?, 28, 28, 1), (?,)), types: (tf.uint8, tf.int64)>

In [5]:
dataset_info.features

FeaturesDict({'image': Image(shape=(28, 28, 1), dtype=tf.uint8), 'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10)})

In [6]:
image_size = 28

In [13]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
base_model = ResNet18((image_size, image_size, 1), weights=None, include_top=False)

In [14]:
base_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 28, 28, 1)    3           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_18 (ZeroPadding2 (None, 34, 34, 1)    0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 14, 14, 64)   3136        zero_padding2d_18[0][0]          
____________________________________________________________________________________________

In [18]:
# images, labels = train_dataset.make_one_shot_iterator().get_next()

In [16]:
n_classes = 10
inputs = keras.Input(shape=(image_size, image_size, 1), name='images')
feature_map = base_model(inputs)
x = keras.layers.GlobalAveragePooling2D()(feature_map)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
cls_model = keras.models.Model(inputs=[inputs], outputs=[output])

In [17]:
cls_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          [(None, 28, 28, 1)]       0         
_________________________________________________________________
model_1 (Model)              (None, 1, 1, 512)         11180611  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 11,185,741
Trainable params: 11,177,803
Non-trainable params: 7,938
_________________________________________________________________


In [20]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(train_dataset, verbose=2, epochs=10, validation_data=test_dataset, validation_steps=10, steps_per_epoch=1000)

Epoch 1/10


W1016 20:57:08.490510 139800664598336 training_arrays.py:289] Your dataset ran out of data; interrupting training. Make sure that your dataset can generate at least `steps_per_epoch * epochs` batches (in this case, 10000 batches). You may need to use the repeat() function when building your dataset.


In [1]:
!pip install albumentations

Looking in indexes: https://pypi.org/simple, https://pypi.cloud.shelfwise.ai/simple
    100% |████████████████████████████████| 102kB 324kB/s a 0:00:01
    100% |████████████████████████████████| 634kB 655kB/s ta 0:00:011
    100% |████████████████████████████████| 28.7MB 898kB/s ta 0:00:011   35% |███████████▌                    | 10.3MB 3.1MB/s eta 0:00:07    85% |███████████████████████████▌    | 24.6MB 1.2MB/s eta 0:00:04    97% |███████████████████████████████▎| 28.0MB 1.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for albumentations ... done
  Stored in directory: /home/mkk/.cache/pip/wheels/94/7d/3b/82e3372b2f6fe2b12635df47fd3ff18bec8df488db0f5475e6
  Running setup.py bdist_wheel for imgaug ... done
  Stored in directory: /home/mkk/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
wise-stitching 0.3.1 has requirement numpy==1.14.3, but you'll have numpy 1.16.4 which is incompatible.
wise-stitching 0.3.1 has

In [36]:
import cv2
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)

AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.2, p=0.5),
    HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20,
                       val_shift_limit=10, p=.9),
    # CLAHE(p=1.0, clip_limit=2.0),
    ShiftScaleRotate(
        shift_limit=0.0625, scale_limit=0.1, 
        rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.8), 
    ToFloat(max_value=255)
])

AUGMENTATIONS_TEST = Compose([
    # CLAHE(p=1.0, clip_limit=2.0),
    ToFloat(max_value=255)
])

<DatasetV1Adapter shapes: ((?, 28, 28, 1), (?,)), types: (tf.uint8, tf.int64)>